<a href="https://colab.research.google.com/github/NamanNYC/Final_Project/blob/main/scrape%26dbdump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initiate Connection to PostgreSQL and RDS**

In [1]:

import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [67.4 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-09-19 15:38:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  4.91MB/s    in 0.2s    

2021-09-19 15:38:06 (4.91 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [7]:
connection_string = 'lmpt-finalproject.coke2w4vs8wf.us-east-2.rds.amazonaws.com'
password = 'LMPTp4ssw0rd'
database_name = 'postgres'

# Configure settings for RDS
mode = "append"
jdbc_url=f"jdbc:postgresql://{connection_string}:5432/{database_name}"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

**Import Links to Data**

In [8]:
import pandas as pd
table_dfs = pd.read_html('https://wildfire.alberta.ca/resources/historical-data/historical-wildfire-database.aspx')
table_df = table_dfs[0]

In [9]:
table_df

,Date Range,Data Download,Data Dictionary
0,2006 - 2018,fires_2006to2018.csv,data_dictionary_2006to2018.pdf
1,1996 – 2005,fires_1996to2005.csv,data_dictionary_1996to2005.pdf
2,1983 – 1995,fires_1983to1995.csv,data_dictionary_1983to1995.pdf
3,1961 – 1982,fires_1961to1982.csv,data_dictionary_1961to1982.pdf


In [10]:
table_df['BaseLink'] = [f'https://wildfire.alberta.ca/resources/historical-data/documents/{link}' for link in table_df['Data Download']]

In [11]:
table_df

,Date Range,Data Download,Data Dictionary,BaseLink
0,2006 - 2018,fires_2006to2018.csv,data_dictionary_2006to2018.pdf,https://wildfire.alberta.ca/resources/historic...
1,1996 – 2005,fires_1996to2005.csv,data_dictionary_1996to2005.pdf,https://wildfire.alberta.ca/resources/historic...
2,1983 – 1995,fires_1983to1995.csv,data_dictionary_1983to1995.pdf,https://wildfire.alberta.ca/resources/historic...
3,1961 – 1982,fires_1961to1982.csv,data_dictionary_1961to1982.pdf,https://wildfire.alberta.ca/resources/historic...


In [17]:
# Import CSVs into Datatbase via a For Loop
from pyspark import SparkFiles
from pyspark.sql import SparkSession

for url, name in zip(table_df['BaseLink'],table_df['Data Download']):
  
    spark = SparkSession.builder.appName(name).config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()
    tablename = name.replace(".csv","")
    spark.sparkContext.addFile(url)
    df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
    df.write.jdbc(url=jdbc_url, table=tablename, mode=mode, properties=config)
    spark.stop()
    print(f'{name} upload completed')

fires_2006to2018.csv upload completed
fires_1996to2005.csv upload completed
fires_1983to1995.csv upload completed
fires_1961to1982.csv upload completed
